# 18 visidom可视化

### tensorboard 工具

### tensorboardX-Pytorch

In [1]:
import  torch
import  torch.nn as nn
import  torch.nn.functional as F
import  torch.optim as optim
from    torchvision import datasets, transforms

In [2]:
from visdom import Visdom

In [3]:
batch_size=200
learning_rate=0.01
epochs=10

In [4]:
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('./data', train=True, download=True,
                   transform=transforms.Compose([
                       transforms.ToTensor(),
                       # transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('./data', train=False, transform=transforms.Compose([
        transforms.ToTensor(),
        # transforms.Normalize((0.1307,), (0.3081,))
    ])),
    batch_size=batch_size, shuffle=True)

In [5]:
class MLP(nn.Module):

    def __init__(self):
        super(MLP, self).__init__()

        self.model = nn.Sequential(
            nn.Linear(784, 200),
            nn.LeakyReLU(inplace=True),
            nn.Linear(200, 200),
            nn.LeakyReLU(inplace=True),
            nn.Linear(200, 10),
            nn.LeakyReLU(inplace=True),
        )

    def forward(self, x):
        x = self.model(x)

        return x

In [6]:
device = torch.device('cuda:0')
net = MLP().to(device)
optimizer = optim.SGD(net.parameters(), lr=learning_rate)
criteon = nn.CrossEntropyLoss().to(device)

## 加入visdom对训练数据进行监视

In [7]:
viz = Visdom()

viz.line([0.], [0.], win='train_loss', opts=dict(title='train loss'))
viz.line([[0.0, 0.0]], [0.], win='test', opts=dict(title='test loss&acc.',
                                                   legend=['loss', 'acc.']))
global_step = 0

In [8]:
for epoch in range(epochs):

    for batch_idx, (data, target) in enumerate(train_loader):
        data = data.view(-1, 28*28)
        data, target = data.to(device), target.cuda()

        logits = net(data)
        loss = criteon(logits, target)

        optimizer.zero_grad()
        loss.backward()
        # print(w1.grad.norm(), w2.grad.norm())
        optimizer.step()

        global_step += 1
        viz.line([loss.item()], [global_step], win='train_loss', update='append')  # 直线数据还是得numpy .item(),image 可以直接tensor。

        if batch_idx % 100 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                       100. * batch_idx / len(train_loader), loss.item()))


    test_loss = 0
    correct = 0
    for data, target in test_loader:
        data = data.view(-1, 28 * 28)
        data, target = data.to(device), target.cuda()
        logits = net(data)
        test_loss += criteon(logits, target).item()

        pred = logits.argmax(dim=1)
        correct += pred.eq(target).float().sum().item()

    viz.line([[test_loss, correct / len(test_loader.dataset)]],
             [global_step], win='test', update='append')
    viz.images(data.view(-1, 1, 28, 28), win='x')
    viz.text(str(pred.detach().cpu().numpy()), win='pred',   # .detach() 返回相同数据的 tensor
             opts=dict(title='pred'))

    test_loss /= len(test_loader.dataset)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))


Train Epoch: 0 [0/60000 (0%)]	Loss: 2.303169
Train Epoch: 0 [20000/60000 (33%)]	Loss: 2.280193
Train Epoch: 0 [40000/60000 (67%)]	Loss: 2.263378

Test set: Average loss: 0.0111, Accuracy: 4115.0/10000 (41%)

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.214029
Train Epoch: 1 [20000/60000 (33%)]	Loss: 2.144128
Train Epoch: 1 [40000/60000 (67%)]	Loss: 1.965252

Test set: Average loss: 0.0092, Accuracy: 5044.0/10000 (50%)

Train Epoch: 2 [0/60000 (0%)]	Loss: 1.826575
Train Epoch: 2 [20000/60000 (33%)]	Loss: 1.712982
Train Epoch: 2 [40000/60000 (67%)]	Loss: 1.481638

Test set: Average loss: 0.0067, Accuracy: 6377.0/10000 (64%)

Train Epoch: 3 [0/60000 (0%)]	Loss: 1.223653
Train Epoch: 3 [20000/60000 (33%)]	Loss: 1.135105
Train Epoch: 3 [40000/60000 (67%)]	Loss: 1.120881

Test set: Average loss: 0.0055, Accuracy: 6692.0/10000 (67%)

Train Epoch: 4 [0/60000 (0%)]	Loss: 1.112903
Train Epoch: 4 [20000/60000 (33%)]	Loss: 0.946192
Train Epoch: 4 [40000/60000 (67%)]	Loss: 1.118770

Test set: Average loss

可视化结果：
<img src='./images/23.png' style="zoom:50%">